#How to sample from the final states:

Use the **predict_final_state** function to sample the final state of the quantum register. \\
This function computes the probabilities for each state and then samples according to these probabilities.

##Gates
**Hadamard Gate (H_matrix)**: Creates superposition in a qubit. \\
**Pauli-X Gate (X_matrix)**: Acts as a NOT gate, flipping a qubit's state. \\
**CNOT Gate**: A two-qubit gate used for entanglement, which flips the target qubit based on the control qubit's state. \\
**CNOT Tensor**: The CNOT matrix is reshaped into a tensor to handle multi-qubit operations in a quantum register. \\

In [18]:
import numpy as np
from scipy.linalg import norm

# Define the quantum gates as matrices
H_matrix = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
X_matrix = np.array([[0, 1], [1, 0]])  # Pauli-X gate (NOT gate)

# CNOT gate represented as a matrix for two qubits
CNOT_matrix = np.array([[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1],
                        [0, 0, 1, 0]])

# Reshape the CNOT matrix into a 4-dimensional tensor for multi-qubit operations
CNOT_tensor = np.reshape(CNOT_matrix, (2, 2, 2, 2))

##Projectors:
These matrices are used for measurement. \\
When you measure a quantum state, you project it onto the basis states, and these matrices correspond to the |0⟩ and |1⟩ measurement outcomes.

In [ ]:
# Define the projectors for measurement
projectors = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]

##Quantum Register:
A class to represent the quantum state and operations on it. \\
**self.num_qubits**: The number of qubits. \\
**self.psi**: The quantum state represented as a tensor. It's initialized to |0⟩ state for all qubits. \\
**self.circuit**: Tracks which gates are applied to each qubit, helping visualize the circuit.

**apply_gate**: This method applies a single-qubit gate to the specified qubit. It uses np.tensordot to apply the matrix multiplication between the gate and the quantum state, and then reorders the axes to maintain proper qubit indexing. \\
**apply_CNOT**: This method applies a CNOT gate between two qubits. Again, np.tensordot is used for the matrix-tensor product, followed by np.moveaxis to maintain qubit order.

In [20]:
# Quantum register class with circuit tracking
class QuantumRegister:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.psi = np.zeros((2,) * num_qubits)  # Initialize the quantum state
        self.psi[(0,) * num_qubits] = 1  # Start in the |0...0⟩ state
        self.circuit = [[] for _ in range(num_qubits)]  # Initialize an empty circuit for each qubit

    # Method to apply a single qubit gate to a specific qubit
    def apply_gate(self, gate, qubit_idx, gate_name):
        self.psi = np.tensordot(gate, self.psi, (1, qubit_idx))  # Apply the gate
        self.psi = np.moveaxis(self.psi, 0, qubit_idx)  # Restore correct axis order
        self.circuit[qubit_idx].append(gate_name)  # Track the gate application

    # Method to apply a CNOT gate between two qubits
    def apply_CNOT(self, control_idx, target_idx):
        self.psi = np.tensordot(CNOT_tensor, self.psi, ((2, 3), (control_idx, target_idx)))
        self.psi = np.moveaxis(self.psi, (0, 1), (control_idx, target_idx))  # Correct axis order
        self.circuit[control_idx].append(f"CNOT(control)")
        self.circuit[target_idx].append(f"CNOT(target)")

    # Method to print the circuit
    def print_circuit(self):
        for qubit_idx, gates in enumerate(self.circuit):
            print(f"Qubit {qubit_idx}: {' -> '.join(gates) if gates else 'No gates applied'}")


    def expectation_value(self, operator):
        # Flatten the statevector to treat it as a vector (instead of a tensor)
        statevector = self.psi.flatten()

        # Compute the conjugate transpose of the statevector
        state_conjugate = np.conjugate(statevector)

        # Perform matrix multiplication: ⟨ψ|O|ψ⟩
        # Step 1: Apply operator to statevector (O|ψ⟩)
        intermediate_result = np.dot(operator, statevector)

        # Step 2: Compute ⟨ψ| * (O|ψ⟩)
        expectation_val = np.dot(state_conjugate, intermediate_result)

        return expectation_val

## Methods
**print_circuit**: Displays the sequence of gates applied to each qubit in the register. \\
**get_state_probability**: Computes the probability of observing a specific quantum state based on the amplitude. \\
**print_state_probabilities**: Iterates over all possible states, printing the probability of each one.

In [ ]:
# Function to apply a projector and return the projected state
def get_state_probability(state, reg):
    index = state
    amplitude = reg.psi[index]  # Get the amplitude of the corresponding state
    probability = np.abs(amplitude) ** 2  # Probability is the square of the amplitude's magnitude
    return probability

# Function to print probabilities of all possible states
def print_state_probabilities(reg):
    num_qubits = reg.num_qubits
    for state_idx in range(2 ** num_qubits):
        state_tuple = tuple(int(x) for x in np.binary_repr(state_idx, width=num_qubits))
        prob = get_state_probability(state_tuple, reg)
        print(f"Probability of state |{''.join(map(str, state_tuple))}⟩: {prob:.4f}")


**predict_final_state**: This function samples the final state of the quantum register after measurement, based on the probabilities of each state. \\
It normalizes the state probabilities and uses np.random.choice to sample according to the probability distribution.

In [ ]:
def predict_final_state(reg):
    num_qubits = reg.num_qubits
    state_probabilities = []
    states = []

    # Gather all possible states and their non-zero probabilities
    for state_idx in range(2 ** num_qubits):
        state_tuple = tuple(int(x) for x in np.binary_repr(state_idx, width=num_qubits))
        prob = get_state_probability(state_tuple, reg)
        if prob > 0:  # Only consider states with non-zero probabilities
            state_probabilities.append(prob)
            states.append(state_tuple)

    # Normalize the probabilities
    state_probabilities = np.array(state_probabilities)
    state_probabilities /= state_probabilities.sum()  # Ensure the probabilities sum to 1

    # Use np.random.choice to randomly select a state based on the non-zero probabilities
    final_state_idx = np.random.choice(len(states), p=state_probabilities)
    collapsed_state = states[final_state_idx]

    print(f"Final collapsed state: |{''.join(map(str, collapsed_state))}⟩")
    return collapsed_state

## Run Simulation
**run_simulation**
: Sets up a quantum circuit for the given number of qubits. \\
It applies an X gate, a Hadamard gate, and CNOT gates between adjacent qubits.

In [14]:
# Function to run the simulation and track gates for circuit printing
def run_simulation(num_qubits):
    reg = QuantumRegister(num_qubits)  # Initialize quantum register

    for i in range(num_qubits - 1):
        reg.apply_gate(X_matrix, i, 'X')
        reg.apply_gate(H_matrix, i, 'H')
        reg.apply_CNOT(i, i + 1)  # Apply CNOT between adjacent qubits

    # Apply X and H gates to the last qubit if necessary
    if num_qubits-1 ==0:
        reg.apply_gate(X_matrix, num_qubits - 1, 'X')
        reg.apply_gate(H_matrix, num_qubits - 1, 'H')

    return reg

##Output
This final block executes the entire simulation. It:


1.   Runs the quantum circuit.
2.   Prints the circuit that was built.
3.   Displays the probabilities of all possible states.
4.   Samples a final state based on the calculated probabilities.


In [15]:
# Example usage
for num_qubits in range(1, 6):
    reg = run_simulation(num_qubits)  # Run the circuit
    reg.print_circuit()  # Print the quantum circuit
    print_state_probabilities(reg)  # Print probabilities of all states
    predict_final_state(reg)  # Predict and print the final collapsed state
    print("\n")



Qubit 0: X -> H
Probability of state |0⟩: 0.5000
Probability of state |1⟩: 0.5000
Final collapsed state: |0⟩


Qubit 0: X -> H -> CNOT(control)
Qubit 1: CNOT(target)
Probability of state |00⟩: 0.5000
Probability of state |01⟩: 0.0000
Probability of state |10⟩: 0.0000
Probability of state |11⟩: 0.5000
Final collapsed state: |11⟩


Qubit 0: X -> H -> CNOT(control)
Qubit 1: CNOT(target) -> X -> H -> CNOT(control)
Qubit 2: CNOT(target)
Probability of state |000⟩: 0.2500
Probability of state |001⟩: 0.0000
Probability of state |010⟩: 0.0000
Probability of state |011⟩: 0.2500
Probability of state |100⟩: 0.2500
Probability of state |101⟩: 0.0000
Probability of state |110⟩: 0.0000
Probability of state |111⟩: 0.2500
Final collapsed state: |011⟩


Qubit 0: X -> H -> CNOT(control)
Qubit 1: CNOT(target) -> X -> H -> CNOT(control)
Qubit 2: CNOT(target) -> X -> H -> CNOT(control)
Qubit 3: CNOT(target)
Probability of state |0000⟩: 0.1250
Probability of state |0001⟩: 0.0000
Probability of state |0010⟩: